In [79]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import TensorDataset, DataLoader, Dataset

import torch.nn.functional as F

import numpy as np


import matplotlib.pyplot as plt

from PIL import Image

import os
import glob

In [89]:
class CustomDataLoader(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        image = torch.unsqueeze(image, 0)
        return image, label

In [81]:
def load_and_resize_image(image_path):
    image = Image.open(image_path)  
    new_size = (144, 144)
    resized_image = image.resize(new_size)
    preprocess = transforms.Compose([
    transforms.ToTensor(),                       # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize pixel values (mean and std values are common for many pre-trained models)
])
    return preprocess(resized_image)

In [82]:
root_directory = "dataset_chat/" 
subdirectories = [d for d in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory, d))]

train_images = []
test_images = []
train_labels = []
test_labels = []

n_classes = 10
i = 0
for subdirectory in subdirectories[:n_classes]:
    print("feu!")
    subdirectory_path = os.path.join(root_directory, subdirectory)
    jpeg_files = glob.glob(os.path.join(subdirectory_path, "*.jpg"))
    
    label = torch.zeros(n_classes)
    label[i] = 1  
    

    for jpeg_file in jpeg_files[:10]:
        tensor_image = load_and_resize_image(jpeg_file)
        train_images.append(tensor_image)
        train_labels.append(label)
    test_images.append(load_and_resize_image(jpeg_files[11]))
    test_labels.append(label)
    
    
    i += 1


feu!
feu!
feu!
feu!
feu!
feu!
feu!
feu!
feu!
feu!


In [92]:
batch_size = 64
learning_rate = 100
num_epochs = 5
train_dataset = CustomDataLoader(train_images, train_labels)
test_dataset = CustomDataLoader(test_images, test_labels)

In [100]:
import torchvision.models as models

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = models.resnet18(pretrained=True, progress=False)     
        
               
        dim_before_fc = self.model.fc.in_features
        print(dim_before_fc)
        self.model.fc = nn.Linear(dim_before_fc, 10)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        x =  self.model(x)
        return self.sigmoid(x)


In [108]:
def eval_model(model, test_dataset, epoch, num_epochs):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_dataset:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            print(predicted)
            print(labels)
            for i in range(len(labels)):
                if predicted == np.argmax(labels):
                    correct += 1

    accuracy = 100 * correct / total
    print(f'Epoch [{epoch + 1}/{num_epochs}], Test Accuracy: {accuracy:.2f}%')

    model.train()

In [101]:
model = SimpleCNN(n_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
    print(epoch)
    for images, labels in train_dataset:
        optimizer.zero_grad()
        outputs = model(images)[0]
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    eval_model(model, test_dataset, epoch, num_epochs)


print('Training and evaluation finished.')


512
0
Epoch [1/5], Test Accuracy: 10.00%
1
Epoch [2/5], Test Accuracy: 10.00%
2
Epoch [3/5], Test Accuracy: 10.00%
3
Epoch [4/5], Test Accuracy: 10.00%
4
Epoch [5/5], Test Accuracy: 10.00%
Training and evaluation finished.


In [109]:
eval_model(model, test_dataset, 0, 0)

tensor([0])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0])
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0])
tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])
tensor([0])
tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])
tensor([0])
tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])
tensor([0])
tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])
tensor([0])
tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])
tensor([0])
tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])
tensor([0])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])
tensor([0])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
Epoch [1/0], Test Accuracy: 10.00%
